In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import groupby
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
pd.options.mode.chained_assignment = None 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random_seed_list=random.sample(range(1, 878), 20)
def standardization (df):
    df_mean=np.nanmean(df)
    df_std=np.nanstd(df)
    df_stdz=(df-df_mean)/df_std
    return(df_stdz)
import classification_daily_SPINUP_Bidirection as clss
import pros_cls_dat_0120_variables_SPINUP_BIdirection as pdc
import shapely.wkt
import captum
from captum.attr import IntegratedGradients, Occlusion, LayerGradCam, LayerAttribution

In [2]:
index_well=pd.read_excel('D:/index_well_KS_NE_TX/NE_TX_KS_well_location_xlsx.xlsx')
well_condition=index_well.iloc[:,-1]
index_well_name=index_well[well_condition.isnull()]['well_no']
columns_well=index_well_name.tolist()
# fold_pth='D:/NE_TX_KS_time_series/ready_state_01-20/'
fold_pth='D:/NE_TX_KS_time_series/ready_state_0120_v4/'
time_var=pd.read_csv('D:/NE_TX_KS_time_series/time_indicator_15May_15Oct.csv', index_col=0)
time_var.index=pd.to_datetime(time_var.index)

In [7]:
len(columns_well)

21

In [4]:
"'configurations'"
# index_well=pd.read_excel('D:/index_well_KS_NE_TX/NE_TX_KS_well_location_xlsx.xlsx')
# index_well_name=index_well.iloc[:,0]
variables=[ 
    'ld_blue', 'ld_green', 'ld_nir', 'ld_red','ld_swir1', 'ld_swir2', 'ld_thermal1',    
        'mod_blue','mod_et', 'mod_green', 'mod_nir', 'mod_red','mod_swir1', 'mod_swir2'  

    ,'noah_mp',
       'tem', 'vpd',
    'pr_5mmthrehold'
    ,'denisarea'    #'agri_area','doy',
]
check_c=['well_level','time','irrilabel','name_col','event_label','pr_label'
         ,'pr_raw','pr_5mmthrehold_raw','vpd_raw','dryspell']#


variables_withy=variables+check_c#'irrigation_RMSE',

time_indicator=['doy']#['year_decimal','month_numerical','June','July',	'August','September']


print(columns_well)

n_step=122
no_class_y=2
split_fraction=0.2
split_fraction_2=0.25
fil_nan=20
start_dat='2008-01-01'
end_dat='2020-12-31'
start_year=2008
end_year=2020

['NE_4', 'NE_6', 'NE_14', 'NE_18', 'NE_21', 'NE_24', 'NE_50', 'NE_53', 'NE_57', 'KS_1', 'KS_4', 'KS_6', 'KS_11', 'KS_12', 'KS_13', 'KS_15', 'KS_20', 'KS_21', 'KS_22', 'KS_23', 'KS_26']


In [10]:
ready_data=pdc.make_sequence(columns_well,fold_pth,time_indicator,start_year, end_year,variables_withy,time_var)


In [12]:
dry_spell_all=[]
for i in ready_data:
    kkk=np.copy(i['pr_raw'])
    kkk[kkk==0]=2
    
    dry_group=[sum(group) for _, group in groupby(kkk)]
    dry_inde=[k_inde for k_inde, group in groupby(kkk)]
    dry_label= [ np.full((int(dry_group[z]/dry_inde[z] )),dry_group[z]/2).data.tolist() if dry_inde[z]==2 else 
                     np.full((int(dry_group[z]/dry_inde[z] )),0).data.tolist() for z in range(len(dry_group))]
    event_fin=list(itertools.chain.from_iterable(dry_label))
    dry_spell_all.append(event_fin)


In [13]:
dry_spell_std=standardization (dry_spell_all)
ready_data_new=[]
for seq in range(len(ready_data)):
    seq_new=ready_data[seq]
    seq_new['dryspell']=dry_spell_std[seq,:]
    ready_data_new.append(seq_new)

In [14]:
data_X,data_y,data_W=pdc.ready_data(ready_data_new,variables, check_c)

In [ ]:
# torch.save(data_X, 'F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/rmv_well_2/data_input.pt')
# torch.save(data_y, 'F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/rmv_well_2/data_target.pt')
# np.save('F:/manuscript_2022_figure/model_scenario/20_ensemble_withoutAGL/rmv_well_2/data_info.npy',data_W)

In [46]:
data_X1=torch.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_input_indexwell_grid.pt')
data_y=torch.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_target_indexwell_grid.pt')
data_W=np.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/data_info_indexwell_grid.npy',allow_pickle=True)

In [47]:
data_X = np.delete(data_X1, -3, axis=-1)

In [48]:
data_W.shape,data_y.shape,data_X.shape

((134, 154, 11), torch.Size([134, 154]), torch.Size([134, 154, 19]))

In [54]:
# figure1b=[pt[0][3] for pt in data_W]
# well_name_arcgis=set(figure1b)
# figures_arcgis=figure_list[figure_list['well_no'].isin(well_name_arcgis)]
# figures_arcgis.to_excel('D:/index_well_KS_NE_TX/final_used_well.xlsx')

In [ ]:
test_info=np.array(data_W)
well_info=test_info[:,17:-15,:]
name=well_info[:,0,3]
ne_out=["NE" in na for na in name]
ks_out=["KS" in na for na in name]
all_out=[True for na in name]

In [ ]:
test_info=np.array(W_te)
well_info=test_info[:,17:-15,:]
name=well_info[:,0,3]
ne_out=["NE" in na for na in name]
ks_out=["KS" in na for na in name]
all_out=[True for na in name]

In [ ]:
y_tr_ks=y_tr[ne_out]

In [ ]:
label_0=y_tr_ks[y_tr_ks==0]
label_1=y_tr_ks[y_tr_ks==1]
label_0.shape, label_1.shape

In [ ]:
2610/(3569+2610)

In [ ]:
para_dict = {
  "n_features": data_X.shape[2],
  "seq_length": data_X.shape[1],
  "hidden_size_1": 64,
  "nb_layers_1":1,
  "drop_out_1":0,
  "drop_out_2":0,
  "learning_rate":0.0005,#1e-4,
    "train_episodes":800,
    "batch_size":32,
    "random_seed":0}

In [9]:
data_X

NameError: name 'data_X' is not defined

In [ ]:
X_trcv, X_te,y_trcv,y_te, W_trcv,W_te=train_test_split(data_X,data_y,data_W,test_size=0.2, random_state=15)
X_tr, X_cv,y_tr,y_cv, W_tr,W_cv=train_test_split(X_trcv,y_trcv,W_trcv,test_size=0.2, random_state=15)
# name_per_rd=[name_index['name_col'][0] for name_index in W_te]
tensor_x_te= torch.from_numpy(np.asarray(X_te).astype(np.float32))
tensor_y_te=torch.tensor(np.asarray(y_te),dtype=torch.int64)

tensor_x_tr= torch.from_numpy(np.asarray(X_tr).astype(np.float32))
tensor_y_tr=torch.tensor(np.asarray(y_tr),dtype=torch.int64)
tensor_x_cv= torch.from_numpy(np.asarray(X_cv).astype(np.float32))
tensor_y_cv=torch.tensor(np.asarray(y_cv),dtype=torch.int64)

In [ ]:
torch.save(data_X, 'F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/data_input.pt')
torch.save(data_y, 'F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/data_target.pt')
np.save('F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/data_info.npy',data_W)
np.save('F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/test_info.npy',W_te)

In [ ]:
tensor_x_tr.shape,tensor_y_cv.shape,tensor_y_te.shape

In [ ]:



out_array=[]
target_array=[]

for rs_ind in range(len(random_seed_list)):
    rs=random_seed_list[rs_ind]
    random_seed=rs
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    para_dict['random_seed']=rs
    para_dict['seq_length']=tensor_x_te.size(1)
    accuracy_cv,accuracy_tr,mv_net,criterion=clss.train_validation_lstm_classification(para_dict,device,tensor_x_tr,tensor_y_tr,tensor_x_cv,tensor_y_cv)
    torch.save(mv_net.state_dict(), 
               'F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/model'+str(rs_ind)+'.pt')
    mv_net.eval()
    output_te = mv_net(tensor_x_te.to(device))
    out_array.append(output_te.cpu().data.numpy())
    target_array.append(tensor_y_te.cpu().data.numpy())

In [ ]:
np.save('F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/rs_info_bi.npy'
,random_seed_list)

In [ ]:
tensor_y_te.shape

# RUN TEST 

In [ ]:
random_seed_list=np.load('F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy/rs_info_bi.npy')
out_array=[]
target_array=[]

for rs_ind in range(len(random_seed_list)):#
    model_load =clss.MV_LSTM(para_dict['n_features'],para_dict['seq_length'], para_dict['hidden_size_1'], 
                    para_dict['nb_layers_1'], para_dict['drop_out_1'],para_dict['drop_out_2'])
    model_load_path='F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy/model'+str(rs_ind)+'.pt'
    model_load.load_state_dict(torch.load(model_load_path))
    model_load.to(device)
    model_int=model_load.eval()
    output_te =model_int(tensor_x_te.to(device))
    out_array.append(output_te.cpu().data.numpy())
    target_array.append(tensor_y_te.cpu().data.numpy())

In [ ]:
out_all=np.array(out_array)
target_all=np.array(target_array)

In [ ]:
len(out_array[0])

In [ ]:
np.save( 'F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/BiLSTMoutput1.npy'
        ,out_all)
np.save('F:/manuscript_2022_figure/model_scenario/indexwell_gridscle_without_doy2/BiLSTMtarget1.npy'
        ,target_all)
# np.save('F:/manuscript_2022_figure/step_attribution_file/20_ensemble_run/models_and_other_outputs/wellname_bi.npy', name_array_all)

# calculate attribution

# accuracy

In [ ]:
out_all= np.load( 'F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/BiLSTMoutput.npy'
        )
target_all=np.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/BiLSTMtarget.npy'
        )
W_te=np.load('F:/manuscript_2022_figure/model_scenario/indexwell_with_gridscale_input/test_info.npy',allow_pickle=True
        )

In [ ]:
m= torch.nn.Softmax(dim=-1)
out_soft=m(torch.Tensor(out_all))


In [ ]:
acc_model=[]
for i in range(20):
    acc_data=[]
    for k in range(out_all.shape[1]):
        out_site=out_all[i,k,17:-15,:]
        out_soft=m(torch.Tensor(out_site))
        out_max=torch.argmax(out_soft, axis=-1)
        target_site=target_all[i,k,17:-15]
        target_site_rmv2=target_site[target_site!=2]
        output_site_rmv2=out_max[target_site!=2].cpu().data.numpy()
        train_acc = np.sum(output_site_rmv2 == target_site_rmv2)
        final_train_acc = train_acc/target_site_rmv2.shape[0]#*target.size(1))
        accu=final_train_acc
        acc_data.append(accu)
    acc_model.append(acc_data)

In [ ]:
len(acc_model[0])

In [ ]:
# name=[i['name_col'][0] for i in W_te]
# year=[int(year_index['time'][0][0:4]) for year_index in W_te]
# name_en=list(np.repeat(name,20))
# year_en=list(np.repeat(year,20))


In [ ]:
acc_all=pd.DataFrame(data=acc_model,columns=W_te[:,0,3])
year=pd.to_datetime(W_te[:,:,1][:,0]).year

In [ ]:
acc_mean_all=pd.DataFrame(acc_all.mean(), columns=['acc'])
acc_mean_all['year']=year

In [ ]:
acc_mean_all.groupby(acc_mean_all.index).mean().sort_values(by=['acc'])

In [ ]:
np.save('F:/manuscript_2022_figure/model_scenario/indexwell_gridscale_v2/test_info.npy', W_te)